In [1]:
# !pip install flet

  Attempting uninstall: watchdog
    Found existing installation: watchdog 2.1.3
    Uninstalling watchdog-2.1.3:
      Successfully uninstalled watchdog-2.1.3
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.


In [117]:
import pandas as pd

h = pd.read_csv('hotels_data.csv')
r = pd.read_csv('restaurants_data.csv')
t = pd.read_csv('tourists_spots.csv')

In [118]:
def get_hotels(city,budget):
    req_h = h[h['City']==city]
    req_h = req_h[['Name of the hotel','Cost','Cluster','sentiment_score']]
    req_h = req_h[req_h['Cost']<=int(budget)]
    res = ''
    if len(req_h)==0:
        res = 'Sorry..! No Hotels match your criteria'
    else:
        for i in list(set(req_h['Cluster'])):
            res += '\n\nSET - '+str(i+1)+' Hotels'
            temp = req_h[req_h['Cluster']==i]
            temp.sort_values('sentiment_score')
            for j in temp.index:
                res += '\n\t\t\t\t\t'+str(temp['Name of the hotel'][j])+'\t\t\t\t\t\t'+str(temp['Cost'][j])
    return res

In [119]:
def get_restaurants(city):
    req_r = r[r['City']==city]
    req_r = req_r[['Name of the hotel','Cluster','sentiment_score']]
    res = ''
    if len(req_r)==0:
        res = 'Sorry..! No Restaurants match your criteria'
    else:
        for i in list(set(req_r['Cluster'])):
            res += '\n\nSET - '+str(i+1)+' Restaurants'
            temp = req_r[req_r['Cluster']==i]
            temp.sort_values('sentiment_score')
            for j in temp.index:
                res += '\n\t\t\t\t\t'+str(temp['Name of the hotel'][j]) 
    return res

In [120]:
def get_tourists(city):
    req_t = t[t['City']==city]
    req_t = req_t[['Name_of_tourist_place','Cluster']]
    res = ''
    if len(req_t)==0:
        res = 'Sorry..! No Tourist Places match your criteria'
    else:
        for i in list(set(req_t['Cluster'])):
            res += '\n\nSET - '+str(i+1)+' Tourist Spots'
            temp = req_t[req_t['Cluster']==i]
            for j in temp.index:
                res += '\n\t\t\t\t\t'+str(temp['Name_of_tourist_place'][j]) 
    return res

In [121]:
import smtplib

FROM = "abhanu92002@gmail.com"
SUBJECT= "Travel Recommendations"
PWD = "jynxcbtycxlmrwpf"

def send_mail(TO,c,b):
    h_text = get_hotels(c,b)
    res_tex = get_restaurants(c)
    ts_tex = get_tourists(c)
    
    h_text = h_text.lstrip('\n')
    res_tex = res_tex.lstrip('\n')
    ts_tex = ts_tex.lstrip('\n')
    
    h_text = h_text.replace('\n','<br>')
    res_tex = res_tex.replace('\n','<br>')
    ts_tex = ts_tex.replace('\n','<br>')
    
    h_text = h_text.replace('\t','&nbsp;')
    res_tex = res_tex.replace('\t','&nbsp;')
    ts_tex = ts_tex.replace('\t','&nbsp;')
    
    TEXT="""
    <h1>Recommended places to visit in""" +c+"""</h1><br>"""+"<h2>Recommeded hotels n your budget</h2>"+h_text+"<br><h2>Recommended restaurants</h2>"+res_tex+"<br><h2>Recommended tourist spots</h2>"+ts_tex
    #Your Message (Even Supports HTML Directly)

    message = f"Subject: {SUBJECT}\nFrom: {FROM}\nTo: {TO}\nContent-Type: text/html\n\n{TEXT}" #This is where the stuff happens

    try:
        server=smtplib.SMTP("smtp.gmail.com",587)
        server.ehlo()
        server.starttls()
        server.login(FROM,PWD)
        server.sendmail(FROM,TO,message)
        server.close()
        print("Successfully sent the mail.")
    except Exception as e:
        print("Failed to send the mail..", e)

In [122]:
import flet
import re
from flet import Page, Text, TextButton, AlertDialog, RadioGroup, Radio, colors, theme, RadialGradient, Dropdown, dropdown, Checkbox, ElevatedButton, TextField, Row, IconButton, icons, Container, Column

def main(page: Page):
    page.title = "Travel Planning Helper"
    page.scroll = "adaptive"
    page.padding = 20
    page.window_always_on_top = True
    page.horizontal_alignment = "center"
    page.bgcolor = colors.PINK_100
    page.scroll = 'adaptive'
    
    mail_sub_text = Text(
        "Do you want an email with the recommended places? ",
        size = 20,
        color=colors.BLACK,
        weight="w400",)
    
    mail_sub = RadioGroup(
        content = Row([
            Radio(value="yes", label="Yes"),
            Radio(value="no", label="No"),]
        )
    )
    
    city = Dropdown(
        label="City",
        hint_text="Choose the city you want to visit",
        options=[
            dropdown.Option("Hyderabad"),
            dropdown.Option("Vijayawada"),
            dropdown.Option("Vishakapatnam"),
            dropdown.Option("Kakinada"),
          ],
        autofocus=True,
        bgcolor = colors.WHITE,
        border_radius = 5,
        filled = True,
        focused_bgcolor = colors.WHITE,
    )
    
    budget = TextField(
        value="0", 
        width=100, 
        focused_bgcolor = colors.WHITE, 
        bgcolor = colors.WHITE, 
        text_size = 15,
        text_align = "center",
        border_radius = 5,
        filled = True,
        read_only=True
    )
    def minus_click(e):
        budget.value = int(budget.value) - 200
        page.update()

    def plus_click(e):
        budget.value = int(budget.value) + 200
        page.update()
    
    email_address = TextField(
                        label="Email address",
                        hint_text = "Enter your email address",
                        width=300, 
                        focused_bgcolor = colors.WHITE, 
                        bgcolor = colors.WHITE, 
                        text_size = 15,
                        text_align = "center",
                        border_radius = 5,
                        filled = True,
                    )
    t = Text(size=20,
            color=colors.BLACK,
            weight="w400",)
    res_cont = Container(
    content = Text())
     
    hotels_c = Checkbox(label="Hotels", value=False)
    restaurants_c = Checkbox(label="Restaurants", value=False)
    tourist_c = Checkbox(label="Tourist Places", value=False)
    
    def close_dlg(e):
        regex = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'
        if(re.search(regex,email_address.value)):
            t.value = f"An email with recommeded places is sent to the entered email address. \nHere are the recommeded places"
            send_mail(email_address.value, city.value, budget.value)
        else:
            t.value = f"Sorry! An incorrect email address is entered\n Here are your recommendations"
        dlg_modal.open = False
        page.update()
         
    dlg_modal = AlertDialog(
              modal=True,
              title=Text("Email Address"),
              content=Column(
                  [
                    Text("Please enter your email address to receive the\n mail with your recommended places"),
                    email_address,
                    ],
                    ),
                  actions=[
                  TextButton("Continue", on_click=close_dlg),
                ],
        #actions_alignment=ft.MainAxisAlignment.END,
        on_dismiss=lambda e: print("Modal dialog dismissed!"),
        )
        
    def cities_data():
        res_head = Text(
                "Recommended Places of City "+city.value,
                size=25,
                color=colors.BLACK,
                weight="w400",
                italic = True,
                text_align = "center",
            )

        c1 = c2 = c3 = 0
        
        #Hotels
        if hotels_c.value == True:
            res_h = get_hotels(city.value,budget.value)
            c1 = Container(
                content=Column(
                    [
                        Text(
                            "Data of Hotels in your budget",
                            size=30,
                            color=colors.BLACK,
                            weight="w400",
                        ),
                        Text(res_h, font_family='Arial',size=16,),
                    ],
                    spacing=10,
                ),
                bgcolor=colors.WHITE,
                padding=5,
                border_radius=5,
            )
            
        #Restaurants
        if restaurants_c.value == True:
            res_r = get_restaurants(city.value)
            c2 = Container(
                    content=Column(
                    [
                        Text(
                            "Restaurants Data",
                            size=30,
                            color=colors.BLACK,
                            weight="w400",
                        ),
                        Text(res_r, font_family='Arial',size=16,),
                    ],
                    spacing=10,
                ),
                bgcolor=colors.WHITE,
                padding=5,
                border_radius=5,
            )
            
        #Tourist Spots
        if tourist_c.value == True:
            res_t = get_tourists(city.value)
            c3 = Container(
                    content=Column(
                    [
                        Text(
                            "Tourist Spots Data",
                            size=30,
                            color=colors.BLACK,
                            weight="w400",
                        ),
                        Text(res_t, font_family='Arial',size=16,),
                    ],
                    spacing=10,
                ),
                bgcolor=colors.WHITE,
                padding=5,
                border_radius=5,
            )
        if c1!=0:
            if c2!=0:
                if c3!=0:
                    return Column([res_head, Row([c1, c2, c3], spacing=10)])
                else:
                    return Column([res_head, Row([c1, c2], spacing=10)])
            else:
                if c3!=0:
                    return Column([res_head,Row([c1, c3], spacing=10)])
                else:
                    return Column([res_head,Row([c1], spacing=10)])
        else:
            if c2!=0:
                if c3!=0:
                    return Column([res_head,Row([c2, c3], spacing=10)])
                else:
                    return Column([res_head,Row([c2], spacing=10)])
            else:
                return Column([res_head,Row([c3], spacing=10)])
    
    def add_data(e):
        if mail_sub.value == "yes":  
            page.dialog = dlg_modal
            dlg_modal.open = True
            page.update()
        else:
            t.value = ''
        res_cont.content = cities_data()
        page.update()

    page.add(
        Text(
            "Travel Planning Helper",
            size=40,
            color=colors.BLACK,
            weight="w400",
            italic = True,
        ),
        city,
        Row(
            [
                Text(
                    "Choose your budget",
                    size=20,
                    color=colors.BLACK,
                    weight="w400",
                ),
                IconButton(icons.REMOVE, on_click=minus_click),
                budget,
                IconButton(icons.ADD, on_click=plus_click),
            ],
            alignment="center",
        ),
        Row(
            [
                Text(
                    "Select required items: ",
                    size=20,
                    color=colors.BLACK,
                    weight="w400",
                ),
                hotels_c,
                restaurants_c,
                tourist_c,
            ],
            alignment = "center",
        ),
        Row(
            [
                mail_sub_text,
                mail_sub,
            ],
        ),
        ElevatedButton(
            text="Get Details",
            bgcolor = colors.WHITE,
            color = colors.BLUE,
            on_click = add_data,
        ),
        t,
        res_cont,
    )

In [123]:
flet.app(target=main)